# Web scraping - players

In [1]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

In [2]:
load_dotenv()

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

I will be scraping from https://www.footballsquads.co.uk/eng/2018-2019/engprem/

In [53]:
df = pd.DataFrame([],columns=["Number", "Name", "Nat","Pos","Height","Weight","Date of Birth", "Birth Place", "Previous Club","first_name","surname","other_names"])

In [54]:
for season in [2018,2019,2020]:
    teams_soup = get_soup(f"https://www.footballsquads.co.uk/eng/2018-2019/engprem/")
    teams = teams_soup.find('table')
    for team in teams.find_all('a')[5:]:
        team_soup = get_soup(f"https://www.footballsquads.co.uk/eng/2018-2019/engprem/{team.text}")
        main_div = team_soup.find(id='main')
        squad_table = main_div.find('table')
        l = []
        for tr in squad_table.find_all('tr')[1:]:
            td = tr.find_all('td')
            row = [tr.text.strip() for tr in td]
            l.append(row)

        squad_df = pd.DataFrame(l, columns=["Number", "Name", "Nat","Pos","Height","Weight","Date of Birth", "Birth Place", "Previous Club"])
        df = pd.concat([df,squad_df],ignore_index=True)

df[["first_name","surname","other_names"]] = df["Name"].str.lower().str.split(' ',2,expand=True) #split name into forename/surname/others
df["date_of_birth"] = pd.to_datetime(df["Date of Birth"], format='%d-%m-%y',errors='coerce').fillna("1900-01-01")
df["football_position"] = df["Pos"].str.lower().str.strip()
df["nationality"] = df["Nat"].str.lower().str.strip()
df["height"] = df["Height"]
df = df[["first_name","surname","other_names","date_of_birth","height","nationality","football_position"]]
df

,first_name,surname,other_names,date_of_birth,height,nationality,football_position
0,petr,čech,None,1982-05-20 00:00:00,1.96,cze,g
1,héctor,bellerín,None,1995-03-19 00:00:00,1.78,esp,d
2,,None,None,1900-01-01,,,
3,mohamed,elneny,None,1992-07-11 00:00:00,1.80,egy,m
4,sokratis,papastathopoulos,None,1988-06-09 00:00:00,1.83,gre,d
...,...,...,...,...,...,...,...
4051,connor,ronan,None,1998-03-06 00:00:00,1.73,irl,m
4052,jack,ruddy,None,1997-05-18 00:00:00,1.85,sco,g
4053,dominic,iorfa,None,1995-07-08 00:00:00,1.88,eng,d
4054,harry,beasley,None,1998-09-19 00:00:00,1.93,eng,g


remove duplicates and any empty rows

In [57]:
players_df = df.loc[df["first_name"].str.len() > 0,:]
players_df.drop_duplicates()

,first_name,surname,other_names,date_of_birth,height,nationality,football_position
0,petr,čech,None,1982-05-20 00:00:00,1.96,cze,g
1,héctor,bellerín,None,1995-03-19 00:00:00,1.78,esp,d
3,mohamed,elneny,None,1992-07-11 00:00:00,1.80,egy,m
4,sokratis,papastathopoulos,None,1988-06-09 00:00:00,1.83,gre,d
5,laurent,koscielny,None,1985-09-10 00:00:00,1.86,fra,d
...,...,...,...,...,...,...,...
1346,léo,bonatini,None,1994-03-28 00:00:00,1.85,bra,f
1347,connor,ronan,None,1998-03-06 00:00:00,1.73,irl,m
1348,jack,ruddy,None,1997-05-18 00:00:00,1.85,sco,g
1349,dominic,iorfa,None,1995-07-08 00:00:00,1.88,eng,d
